# Laboratório 6 - EXTRA

# **Cabeçalho**

Aluno 1 - NUSP - Graduação/Pós - Período

Aluno 2 - NUSP - Graduação/Pós - Período

#### O Banco de dados *Default*

Vamos utilizar um banco de dados conhecido, o *Default*. Nele, existem características de pessoas que são ou não inadimplentes.

In [ ]:
import pandas as pd

# URL do arquivo Excel raw no GitHub
url = "https://github.com/JWarmenhoven/ISLR-python/raw/master/Notebooks/Data/Default.xlsx"

# Carrega o arquivo Excel em um DataFrame
df = pd.read_excel(url)

df.head()

Vamos importar os pacotes necessários e fazer novamente o one-hot encoding

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression # Modelo de regressão logística do sklearn
from sklearn.model_selection import train_test_split # Função para dividir o banco entre treino e teste
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # Algumas métricas e relatórios

In [ ]:
# Converte as variáveis categóricas em numéricas usando codificação one-hot
df_encoded = pd.get_dummies(df, columns=['default', 'student'], drop_first=True, dtype = int)

df_encoded

Dividir em features e target

In [ ]:
# Divide os dados em features (X) e target (y)
X = df_encoded[['balance', 'income', 'student_Yes']]
y = df_encoded['default_Yes']

In [ ]:
# Desempacotando cada parte com o train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X, # Vetor de variáveis explicativas
                                                        y, # Target (Ou Var. Dep.)
                                                        test_size=0.2, # Porcentagem que vai ficar para teste (20%)
                                                        random_state=42) # Seed para garantir a estabilidade dos resultados

print(X_treino)

#### Linear Discriminant Analysis (LDA)

Como talvez tenham percebido, treinar um modelo qualquer, especialmente os menos computacionalmente intensivos, não é extremamente complicado. Para o nosso interesse, é mais importante entender a intuição geral do modelo, além de suas vantagens e desvantagens, sempre tentando também comparar eles com outros modelos. Vamos agora para o modelo LDA, ou Linear Discriminant Analysis.

O LDA é um método de análise estatística usado para encontrar a melhor combinação linear de características (variáveis) que maximiza a separação entre duas ou mais classes em um conjunto de dados. Em outras palavras, o LDA ajuda a encontrar um "caminho" ou projeção das características que torna mais fácil distinguir entre diferentes grupos de dados.

Aqui estão os passos básicos de como o LDA funciona:

* Calculando as Médias: Para cada classe, o LDA calcula a média das características. Isso significa que ele encontra o valor médio de cada característica para cada grupo de dados. Isso é chamado de "vetor médio" para cada classe.

* Calculando a Dispersão: O LDA também calcula a dispersão ou variabilidade das características dentro de cada classe. Isso ajuda a medir o quão espalhados estão os pontos de dados dentro de cada grupo.

* Encontrando a Melhor Projeção: O objetivo do LDA é encontrar a projeção (combinação linear das características) que maximize a separação entre as médias das classes e minimize a dispersão dentro de cada classe. Em outras palavras, ele tenta encontrar um "caminho" ao longo do qual os dados de cada classe estão mais agrupados e separados dos dados das outras classes.

Em resumo, o LDA é uma técnica que ajuda a encontrar a melhor maneira de separar dados em diferentes grupos com base em suas características. É útil em tarefas de classificação, como reconhecimento de padrões e detecção de padrões em dados multidimensionais.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # Importando o modelo do SKlearn

Como Já separamos anteriormente os dados de treino e teste, vamos já partir para o treino do modelo:

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_treino, y_treino)

Prevendo no banco de teste:

In [ ]:
y_pred_lda = lda.predict(X_teste)

y_pred_lda

Agora, vamos avaliar o modelo

In [ ]:
accuracy = accuracy_score(y_teste, # Valores Reais
                          y_pred_lda) # Valores preditos pelo modelo LDA

print(f"Acurácia: {accuracy}")


A acurácia geral foi minimamente maior do que a da Regressão Logística (96,8% contra 96,4%). Vamos ver a matriz de confusão:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


mat_conf_lda = confusion_matrix(y_teste, y_pred_lda)


# Defina os rótulos das classes
class_names = ['Adimplente', 'Inadimplente']

# Crie um gráfico de matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(mat_conf_lda, # Nossa matriz gerada pelo sklearn
            annot=True,
            fmt='d', # Dígitos completos
            cmap='Blues', # Cor da matriz
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()


O problema na classe de inadimplentes permanece, ainda só 11 de 69 foram classificados corretamente. Podemos confirmar isso olhando para o relatório de classificação:

In [ ]:
rel_class = classification_report(y_teste, y_pred_lda)

print(f"Relatório de Classificação:\n{rel_class}")

Aqui, podemos ver que, mesmo que haja uma maior precisão da classe minoritária (inadimplentes) ainda temos um problema enorme com o recall, gerando uma alta taxa de falsos negativos. Isto é, inadimplentes que estão sendo classificados como adimplentes. Também é sempre importante olhar para o F1-score, que é a média harmonica de precision e recall. Aqui, vemos que o F1-score para a classe 1 (inadimplentes) é baixíssimo (0,26), lembrando que todas estas medidas variam de 0 a 1.

#### Quadratic Discriminant Analysis (QDA)

O QDA é um método estatístico que, assim como o LDA, é usado para separar dados em diferentes classes com base em suas características. No entanto, o QDA assume que as classes têm diferentes matrizes de covariância. Agora, vamos quebrar isso em partes:

* Matrizes de Covariância Diferentes: O QDA parte do pressuposto de que as diferentes classes em seus dados podem ter dispersões diferentes em várias direções. Isso significa que as variâncias e covariâncias (como a relação entre duas características) entre as classes podem ser diferentes.

* Classificação com Superfícies Não-Lineares: Devido à suposição de matrizes de covariância diferentes, o QDA é capaz de modelar relações mais complexas entre as classes. Isso significa que ele pode capturar fronteiras de decisão não-lineares, permitindo maior flexibilidade na classificação. (Nota: Lembre-se que modelos mais flexíveis tem suas vantagens e desvantagens).

* Processo de Treinamento: O treinamento do QDA envolve calcular as matrizes de covariância para cada classe. Isso é feito estimando a matriz de covariância de cada classe com base nos dados de treinamento. Em seguida, o modelo usa essas informações para calcular as probabilidades de pertencimento de uma nova amostra a cada classe.

* Tomada de Decisões: Quando você deseja classificar uma nova amostra de dados, o QDA calcula a probabilidade de pertencimento a cada classe com base nas matrizes de covariância estimadas e usa essas probabilidades para fazer a classificação final. A classe com a maior probabilidade é a classe prevista.

Em resumo, o QDA é uma técnica de classificação que leva em consideração a variabilidade das classes, permitindo que o modelo se adapte a relações não-lineares entre os dados. Isso pode ser útil quando as classes têm diferentes padrões de dispersão e quando as fronteiras de decisão não são estritamente lineares.

1. Importando o modelo:

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

2. Criando o objeto e treinando o modelo:

In [ ]:
# Crie e treine o modelo QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_treino, y_treino)

3. Prevendo no banco de teste

In [ ]:
# Faça previsões no conjunto de teste
y_pred_qda = qda.predict(X_teste)

y_pred_qda

4. Avaliando o desempenho do modelo:

Agora, tente pensar sozinho um pouco sobre a performance do modelo, e compare os resultados com os outros dois modelos.

a) Acurácia

In [ ]:
# Avalie o desempenho do modelo
accuracy = accuracy_score(y_teste, y_pred_qda)

print(f"Acurácia: {accuracy}")

b) Matriz de Confusão

In [ ]:
mat_conf_qda = confusion_matrix(y_teste, y_pred_qda)


# Defina os rótulos das classes
class_names = ['Adimplente', 'Inadimplente']

# Crie um gráfico de matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(mat_conf_qda, # Nossa matriz gerada pelo sklearn
            annot=True,
            fmt='d', # Dígitos completos
            cmap='Blues', # Cor da matriz
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()

c) Relatório de Classificação

In [ ]:
rel_class = classification_report(y_teste, y_pred_qda)

print(f"Relatório de Classificação:\n{rel_class}")

Agora, iremos para os dois últimos modelos que veremos nesta aula: Naive Bayes e K-Neighbors:

#### Naive-Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB # Modelo Naive-Bayes

# Crie e treine o modelo Naive Bayes (Gaussian Naive Bayes)
nb_model = GaussianNB()
nb_model.fit(X_treino, y_treino)

# Faça previsões no conjunto de teste
y_pred_NB = nb_model.predict(X_teste)

# Imprimindo o array de predições

y_pred_NB


a) Accuracy

In [ ]:
accuracy = accuracy_score(y_teste, y_pred_NB)

print(f"Acurácia: {accuracy}")

b) Confusion Matrix

In [ ]:
mat_conf_NB = confusion_matrix(y_teste, y_pred_NB)


# Defina os rótulos das classes
class_names = ['Adimplente', 'Inadimplente']

# Crie um gráfico de matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(mat_conf_NB, # Nossa matriz gerada pelo sklearn
            annot=True,
            fmt='d', # Dígitos completos
            cmap='Blues', # Cor da matriz
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()

#### K-Nearest Neighbors (KNN)

A ideia central do KNN é que os objetos (ou pontos de dados) que são semelhantes tendem a estar próximos uns dos outros em um espaço de características. Portanto, o KNN funciona com base na proximidade entre os pontos de dados em um espaço de características.


2. Parâmetro K:

O "K" em KNN representa o número de vizinhos mais próximos que serão considerados para tomar uma decisão. Por exemplo, se K=3, o algoritmo considerará os três vizinhos mais próximos de um ponto de dados para tomar uma decisão.
3. Classificação com KNN:

Para classificação, o KNN determina a classe de um novo ponto de dados com base na classe da maioria dos K vizinhos mais próximos.
Por exemplo, se a maioria dos K vizinhos mais próximos de um novo ponto de dados pertence à classe "A", o KNN classificará o novo ponto como "A".
4. Regressão com KNN:

Para regressão, o KNN calcula a média (ou outra medida estatística) dos valores-alvo dos K vizinhos mais próximos e usa esse valor para fazer previsões.
Por exemplo, se estamos prevendo o preço de uma casa, o KNN calculará a média dos preços das K casas mais próximas para fazer a previsão.
5. Métrica de Distância:

O KNN usa uma métrica de distância (geralmente a distância Euclidiana) para medir a proximidade entre pontos de dados no espaço de características.
A métrica de distância determina como os pontos são comparados em termos de proximidade.
6. Escolha de K:

A escolha adequada de K é importante. Um valor muito pequeno de K pode levar a um modelo muito sensível a outliers (muito flexível), enquanto um valor muito grande de K pode tornar o modelo menos sensível a padrões sutis nos dados (muito rígido). Uma boa ideia é a de testar varios valores e ver qual performa melhor no banco de teste.

7. Treinamento do Modelo:

No treinamento, o KNN simplesmente armazena todos os pontos de dados com seus rótulos em uma estrutura de dados para que eles possam ser usados para previsões futuras.

8. Previsões:

Para fazer uma previsão com o KNN, você encontra os K vizinhos mais próximos do novo ponto de dados usando a métrica de distância.
Em seguida, toma uma decisão com base na maioria das classes (classificação) ou na média dos valores-alvo (regressão) dos vizinhos mais próximos.

Aplicações:

O KNN é usado em uma variedade de tarefas, como classificação de documentos, recomendação de produtos, diagnóstico médico, detecção de anomalias e muito mais.
Em resumo, o KNN é um método de aprendizado de máquina que faz previsões com base na proximidade entre pontos de dados em um espaço de características. Ele é simples de entender e implementar, mas a escolha apropriada de K e da métrica de distância é fundamental para seu desempenho.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # Importando o KNN

# Crie e treine o modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=3)  # Você pode ajustar o número de vizinhos (K) aqui
knn_model.fit(X_treino, y_treino)


# Faça previsões no conjunto de teste
y_pred_knn = knn_model.predict(X_teste)

y_pred_knn


In [ ]:
# Avalie o desempenho do modelo
accuracy = accuracy_score(y_teste, y_pred_knn)

print(f"Acurácia: {accuracy}")

In [ ]:
mat_conf_KNN = confusion_matrix(y_teste, y_pred_knn)


# Defina os rótulos das classes
class_names = ['Adimplente', 'Inadimplente']

# Crie um gráfico de matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(mat_conf_KNN, # Nossa matriz gerada pelo sklearn
            annot=True,
            fmt='d', # Dígitos completos
            cmap='Blues', # Cor da matriz
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
report = classification_report(y_teste, y_pred_knn)

print(f"Relatório de Classificação:\n{report}")

## Exercícios

### Banco de Dados

1- Agora é sua vez, vamos pegar um outro banco de dados. Importe o banco de dados "Healthcare-Diabetes"

In [ ]:
url = "https://raw.githubusercontent.com/PedroSchmalz/FLP0478/refs/heads/main/book/aulas/aula6/dataset/Healthcare-Diabetes.csv"

In [ ]:
# Imprima o banco aqui

No seguinte kaggle ([link](https://www.kaggle.com/datasets/nanditapore/healthcare-diabetes/)) você encontra mais informações sobre o banco.

2 - Faça uma matriz de correlação das variáveis independentes. Há alguma variável que pareça ser muito correlacionada com outra (muito próximo de 1)?

3 - Separe as colunas em um vetor X de features e o target (Outcome). Também separe o banco entre treino e teste (com 20% para teste).

### Regressão Logística

4 - Agora, treine um modelo de regressão logística.

5 - Faça a predição dos valores do banco de teste e imprima o array de predições.

a) Apresente a acurácia:

b) Apresente a matriz de confusão

c) Por fim, apresente o relatório de classificação

d) O modelo foi bem? Comente sobre

Digite Sua resposta aqui

e) Agora, vamos pegar esses valores do relatório de classificação de forma separada, isso vai ser útil depois. Pegue precision, recall e f1-score e salve-os em uma variável com nome adequado para depois. Imprima os valores e veja se batem com o do relatório de classificação. Dica: [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)

In [ ]:
# Dica:

from sklearn.metrics import precision_recall_fscore_support

resultados = precision_recall_fscore_support(y_teste, y_pred, average=None, labels=[0, 1])



### LDA

6 - Agora, treine um modelo LDA e faça as predições no banco de teste.

a) Apresente a acurácia

b) Apresente a matriz de confusão:

c) Por fim, o relatório de classificação. Os resultados são melhores do que o da regressão logística? Comente sobre.

d) Novamente, pegue os valores separados e os salve em um novo dataframe.

e) Junte com o banco que você criou no exercício 5-e

### QDA

7 - Treine um modelo QDA e faça as predições no banco de teste

a) Apresente o relatório de classificação

b) Salve precisão, recall e f1-score em um dataframe

c) Junte com o banco de dados de resultados por modelo, criado no exercício 6-e

### NB

8 - Treine um Modelo Naive-bayes

a) Imprima o relatório de classificação

b) Salve precision, recall e f1-score em um dataframe separado

c) Junte com o dataframe de resultados dos outros modelos

### KNN

9 - Por fim, treine também um modelo KNN

a) Imprima o relatório de classificação.

b) Salve precision, recall e f1-score em um dataframe separado

c) Por fim, junte estes resultados com o dataframe completo de resultados

Comparação dos modelos

10 - Pegue o dataframe criado em 9-c e drope as colunas de suporte. Depois, transforme do formato wide para o formato long.

11 - Filtre somente os valores referentes à precisão e faça um gráfico de barras comparando os resultados desta métrica.

12 - Qual modelo se saiu melhor nesta métrica?

Resposta:

13 - Faça o mesmo do exercício 11 com o Recall (Revocação)

14 - Qual modelo tem a menor taxa de falsos negativos (maior recall)? Os resultados são parecidos até na classe minoritária?

Resposta:

15 - Por fim, compare os f1-scores da mesma forma.

16 - Qual modelo obteve melhor desempenho nesta tarefa?

Resposta: